## GPU Puzzles in CUDA C++
By Devin Shah - [@devinshah16](https://twitter.com/DevinShah16)

Puzzles adapted from [Sasha Rush](http://rush-nlp.com/)

GPUs are pretty cool.

This notebook is a bit more of an advanced attempt to teach GPU programming interactively. Instead of using Python bindings (through Numba), we will be directly working with CUDA C++ bindings. In this notebook, we will just be focusing on the kernels, but in a later video, I will walk through how to instantiate the kernels, which is a bit harder than using Numba's built in executor.

Be careful with pointers and dereferncing. All of these kernels do not need complicated technques; however, when we implement the kernel executors (coming soon), there are some complex techniques.

I recommend doing Sasha's notebook first, as the visualization are much clearer and will help build intuition.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/srush/GPU-Puzzles/blob/main/GPU_puzzlers.ipynb)

Make your own copy of this notebook in Colab, turn on GPU mode in the settings (`Runtime / Change runtime type`, then set `Hardware accelerator` to `GPU`), and
then get to coding. ***You might get a warning saying that the GPU is not being used, but it is in fact being used. Ignore this warning. If using a free version, be careful of quotas.***


Read the [CUDA C++ bindings guide ](https://docs.nvidia.com/cuda/pdf/CUDA_C_Programming_Guide.pdf)

To test these, there is a single test case that has been created in the executors (in the gh repo). It runs on assertion statements, so your kernel will fail if the assertion statements fail. A compute sanitizer (developed by NVIDIA) is also run on your kernel so that you can debug memory issues and out of bounds issues. This is particularly helpful for shared memory.

In [1]:
!git clone https://github.com/dshah3/GPU-Puzzles.git


Cloning into 'GPU-Puzzles'...
remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 220 (delta 124), reused 122 (delta 111), pack-reused 75
Receiving objects: 100% (220/220), 1019.94 KiB | 5.02 MiB/s, done.
Resolving deltas: 100% (154/154), done.


In [2]:
%cd GPU-Puzzles/GPU_puzzlers_exec

/content/GPU-Puzzles/GPU_puzzlers_exec


Make sure `nvcc` is installed. If it is not, this notebook will not work.

In [8]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


## Puzzle 1: Map

Implement a "kernel" (GPU function) that adds 10 to each position of vector `A`
and stores it in vector `C`.  You have 1 thread per position.

**Warning** This code looks like C++ but it is really CUDA C++! You have to be careful; for example, C++ supports indexing arrays like so: `A[i][j]`, but CUDA C++ allows for 1D indexing only, like so: `A[i * size + j]`.
The puzzles only require doing simple operations, basically
+, *, simple array indexing, for loops, and if statements.
You are allowed to use local variables.
If you get an
error it is probably because you did something fancy :).

In [9]:
%%writefile map_kernel.cu
#include <cuda_runtime.h>

__global__ void ScalarAdd(float* A, float* C) {
  int i = threadIdx.x;

  /// CODE HERE (approx 1 line) ///

}

Overwriting map_kernel.cu


In [10]:
!nvcc -c -o map_runner.o map_runner.cu
!nvcc -c -o map_kernel.o map_kernel.cu
!nvcc -o map map_runner.o map_kernel.o
!./map
!compute-sanitizer ./map

Scalar addition is successful!
========= COMPUTE-SANITIZER
Scalar addition is successful!
========= ERROR SUMMARY: 0 errors


## Puzzle 2 - Zip
Implement a kernel that adds together each position of `A` and `B` and stores it in `C`. You have 1 thread per position.

In [11]:
%%writefile zip_kernel.cu
#include <cuda_runtime.h>

__global__ void VecAdd(float* A, float* B, float* C) {
  int i = threadIdx.x;

  /// CODE HERE (approx 1 line) ///

}

Writing zip_kernel.cu


In [12]:
!nvcc -c -o zip_runner.o zip_runner.cu
!nvcc -c -o zip_kernel.o zip_kernel.cu
!nvcc -o zip zip_runner.o zip_kernel.o
!./zip
!compute-sanitizer ./zip

Vector addition successful!
========= COMPUTE-SANITIZER
Vector addition successful!
========= ERROR SUMMARY: 0 errors


## Puzzle 3 - Guards

Implement a kernel that adds 10 to each position of `A` and stores it in `C`.
You have more threads than positions.

In [13]:
%%writefile guards_kernel.cu
#include <cuda_runtime.h>

__global__ void Guards(float* A, float* C, int size) {
  int i = threadIdx.x;

  /// CODE HERE (approx 3 lines) ///

}

Writing guards_kernel.cu


In [14]:
!nvcc -c -o guards_runner.o guards_runner.cu
!nvcc -c -o guards_kernel.o guards_kernel.cu
!nvcc -o guards guards_runner.o guards_kernel.o
!./guards
!compute-sanitizer ./guards

Guards successful!
========= COMPUTE-SANITIZER
Guards successful!
========= ERROR SUMMARY: 0 errors


## Puzzle 4 - Map 2D

Implement a kernel that adds 10 to each position of `A` and stores it in `C`.
Input `A` is 2D and square. You have more threads than positions.

1D indexing doesn't work for 2D arrays in CUDA C++. You can calculate the index from i and j by computing `i * size + j`.

In [15]:
%%writefile map2d_kernel.cu
#include <cuda_runtime.h>

__global__ void Map2D(float* A, float* C, float size) {
  int local_i = threadIdx.x;
  int local_j = threadIdx.y;

  /// CODE HERE (approx 4 lines) ///

}

Writing map2d_kernel.cu


In [16]:
!nvcc -c -o map2d_runner.o map2d_runner.cu
!nvcc -c -o map2d_kernel.o map2d_kernel.cu
!nvcc -o map2d map2d_runner.o map2d_kernel.o
!./map2d
!compute-sanitizer ./map2d

2D mapping successful
========= COMPUTE-SANITIZER
2D mapping successful
========= ERROR SUMMARY: 0 errors


## Puzzle 5 - Broadcast

Implement a kernel that adds `A` and `B` and stores it in `C`.
Inputs `A` and `B` are vectors. You have more threads than positions.

In [17]:
%%writefile broadcast_kernel.cu
#include <cuda_runtime.h>

__global__ void Broadcast(float* A, float* B, float* C, int size) {
  int local_i = threadIdx.x;
  int local_j = threadIdx.y;

  /// CODE HERE (approx 4 lines) ///

}

Writing broadcast_kernel.cu


In [18]:
!nvcc -c -o broadcast_runner.o broadcast_runner.cu
!nvcc -c -o broadcast_kernel.o broadcast_kernel.cu
!nvcc -o broadcast broadcast_runner.o broadcast_kernel.o
!./broadcast
!compute-sanitizer ./broadcast

Broadcast successful
========= COMPUTE-SANITIZER
Broadcast successful
========= ERROR SUMMARY: 0 errors


## Puzzle 6 - Blocks

Implement a kernel that adds 10 to each position of `A` and stores it in `C`.
You have fewer threads per block than the size of `A`.

*Tip: A block is a group of threads. The number of threads per block is limited, but we can
have many different blocks. Variable `cuda.blockIdx` tells us what block we are in.*

In [23]:
%%writefile blocks_kernel.cu
#include <cuda_runtime.h>

__global__ void Blocks(float* A, float* C, float size) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;

  /// CODE HERE (approx 3 lines) ///

}

Overwriting blocks_kernel.cu


In [24]:
!nvcc -c -o blocks_runner.o blocks_runner.cu
!nvcc -c -o blocks_kernel.o blocks_kernel.cu
!nvcc -o blocks blocks_runner.o blocks_kernel.o
!./blocks
!compute-sanitizer ./blocks

Blocks successful!
========= COMPUTE-SANITIZER
Blocks successful!
========= ERROR SUMMARY: 0 errors


## Puzzle 7 - Blocks 2D

Implement the same kernel in 2D.  You have fewer threads per block
than the size of `A` in both directions.

In [25]:
%%writefile map2d_block_kernel.cu
#include <cuda_runtime.h>

__global__ void Map2DBlock(float* A, float* C, float size) {
  int local_i = blockDim.x * blockIdx.x + threadIdx.x;
  int local_j = blockDim.y * blockIdx.y + threadIdx.y;

  /// CODE HERE (approx 4 lines) ///

}

Writing map2d_block_kernel.cu


In [27]:
!nvcc -c -o map2d_block_runner.o map2d_block_runner.cu
!nvcc -c -o map2d_block_kernel.o map2d_block_kernel.cu
!nvcc -o map2d_block map2d_block_runner.o map2d_block_kernel.o
!./map2d_block
!compute-sanitizer ./map2d_block

2D mapping successful
========= COMPUTE-SANITIZER
2D mapping successful
========= ERROR SUMMARY: 0 errors


## Puzzle 8 - Shared

Implement a kernel that adds 10 to each position of `A` and stores it in `C`.
You have fewer threads per block than the size of `A`.

**Warning**: Each block can only have a *constant* amount of shared
 memory that threads in that block can read and write to. This needs
 to be a literal constant not a variable. After writing to
 shared memory you need to call `__syncthreads();` to ensure that
 threads do not cross.

In [28]:
%%writefile shared_kernel.cu
#include <cuda_runtime.h>

__global__ void Shared(float* A, float* C, float size) {
  extern __shared__ float sharedMem[];

  int i = blockDim.x * blockIdx.x + threadIdx.x;
  int local_i = threadIdx.x;

  /// CODE HERE (approx 7 lines) ///

}

Writing shared_kernel.cu


In [29]:
!nvcc -c -o shared_runner.o shared_runner.cu
!nvcc -c -o shared_kernel.o shared_kernel.cu
!nvcc -o shared shared_runner.o shared_kernel.o
!./shared
!compute-sanitizer ./shared

Shared successful!
========= COMPUTE-SANITIZER
Shared successful!
========= ERROR SUMMARY: 0 errors


## Puzzle 9 - Pooling

Implement a kernel that sums together the last 3 position of `A` and stores it in `C`.
You have 1 thread per position.

In [30]:
%%writefile pooling_kernel.cu
#include <cuda_runtime.h>

__global__ void Pooling(float* A, float* C, float size) {
  extern __shared__ float sharedMem[];
  int i = blockDim.x * blockIdx.x + threadIdx.x;
  int local_i = threadIdx.x;

  /// CODE HERE (approx 7 lines) ///

}

Writing pooling_kernel.cu


In [31]:
!nvcc -c -o pooling_runner.o pooling_runner.cu
!nvcc -c -o pooling_kernel.o pooling_kernel.cu
!nvcc -o pooling pooling_runner.o pooling_kernel.o
!./pooling
!compute-sanitizer ./pooling

Pooling successful!
========= COMPUTE-SANITIZER
Pooling successful!
========= ERROR SUMMARY: 0 errors


## Puzzle 10 - Dot Product

Implement a kernel that computes the dot-product of `A` and `B` and stores it in `C`.
You have 1 thread per position.

In [32]:
%%writefile dotproduct_kernel.cu
#include <cuda_runtime.h>

__global__ void DotProduct(float* A, float* B, float* C, float size) {
  extern __shared__ float sharedMem[];
  int i = blockDim.x * blockIdx.x + threadIdx.x;
  int local_i = threadIdx.x;

  /// CODE HERE (approx 11 lines) ///

}

Writing dotproduct_kernel.cu


In [33]:
!nvcc -c -o dotproduct_runner.o dotproduct_runner.cu
!nvcc -c -o dotproduct_kernel.o dotproduct_kernel.cu
!nvcc -o dotproduct dotproduct_runner.o dotproduct_kernel.o
!./dotproduct
!compute-sanitizer ./dotproduct

Dot product successful!
========= COMPUTE-SANITIZER
Dot product successful!
========= ERROR SUMMARY: 0 errors


## Puzzle 11 - 1D Convolution

Implement a kernel that computes a 1D convolution between `A` and `B` and stores it in `C`.
You need to handle the general case.

The shared memory is initialized to be enough to cover what is needed. In the kernel, the shared memory needs to be split into two different shared memories: `shared_a` and `shared_b`. The sizes of the shared memory will be clear as you develop the kernel.

In [36]:
%%writefile 1dconv_kernel.cu
#include <cuda_runtime.h>

const int TPB = 8;
const int MAX_CONV = 4;
const int TPB_MAX_CONV = TPB + MAX_CONV;

__global__ void Conv1D(float* A, float* B, float* C, int a_size, int b_size) {
  extern __shared__ float sharedMem[];
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int local_i = threadIdx.x;

  float* shared_a = ;
  float* shared_b = ;

  /// CODE HERE (approx 25 lines) ///

}

Overwriting 1dconv_kernel.cu


In [37]:
!nvcc -c -o 1dconv_runner.o 1dconv_runner.cu
!nvcc -c -o 1dconv_kernel.o 1dconv_kernel.cu
!nvcc -o 1dconv 1dconv_runner.o 1dconv_kernel.o
!./1dconv
!compute-sanitizer ./1dconv

1dconv_runner.cu(9): warning #177-D: variable "TPB_MAX_CONV" was declared but never referenced

1D Convolution successful!
========= COMPUTE-SANITIZER
1D Convolution successful!
========= ERROR SUMMARY: 0 errors


## Puzzle 12 - Prefix Sum

Implement a kernel that computes a sum over `A` and stores it in `C`.
If the size of `A` is greater than the block size, only store the sum of
each block.
We will do this using the [parallel prefix sum](https://en.wikipedia.org/wiki/Prefix_sum) algorithm in shared memory.
That is, each step of the algorithm should sum together half the remaining numbers.
Follow this diagram:

<img src="https://user-images.githubusercontent.com/35882/178757889-1c269623-93af-4a2e-a7e9-22cd55a42e38.png" width="400">

In [38]:
%%writefile prefixsum_kernel.cu
#include <cuda_runtime.h>

__global__ void PrefixSum(float* A, float* C, int size) {
  extern __shared__ float cache[];
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int local_i = threadIdx.x;

  /// CODE HERE (approx 14 lines) ///

}

Writing prefixsum_kernel.cu


In [39]:
!nvcc -c -o prefixsum_runner.o prefixsum_runner.cu
!nvcc -c -o prefixsum_kernel.o prefixsum_kernel.cu
!nvcc -o prefixsum prefixsum_runner.o prefixsum_kernel.o
!./prefixsum
!compute-sanitizer ./prefixsum

Prefix sum successful!
========= COMPUTE-SANITIZER
Prefix sum successful!
========= ERROR SUMMARY: 0 errors


## Puzzle 13 - Axis Sum

Implement a kernel that computes a sum over each column of `A` and stores it in `C`.

In [40]:
%%writefile axis_sum_kernel.cu
#include <cuda_runtime.h>

__global__ void AxisSum(float* A, float* C, int size) {
  extern __shared__ float cache[];
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int local_i = threadIdx.x;
  int batch = blockIdx.y;

  /// CODE HERE (approx 14 lines) ///


}

Writing axis_sum_kernel.cu


In [41]:
!nvcc -c -o axis_sum_runner.o axis_sum_runner.cu
!nvcc -c -o axis_sum_kernel.o axis_sum_kernel.cu
!nvcc -o axis_sum axis_sum_runner.o axis_sum_kernel.o
!./axis_sum
!compute-sanitizer ./axis_sum

Axis sum successful!
========= COMPUTE-SANITIZER
Axis sum successful!
========= ERROR SUMMARY: 0 errors


## Puzzle 14 - Matrix Multiply!

Implement a kernel that multiplies square matrices (with the same size) `A` and `B` and
stores the result in `C`.

*Tip: The most efficient algorithm here will copy a block into
 shared memory before computing each of the individual row-column
 dot products. This is easy to do if the matrix fits in shared
 memory.  Do that case first. Then update your code to compute
 a partial dot-product and iteratively move the part you
 copied into shared memory.*

In [42]:
%%writefile matmul_kernel.cu
#include <cuda_runtime.h>

const int TPB = 3;

__global__ void Matmul(float* A, float* B, float* C, int size) {
  extern __shared__ float sharedMem[];

  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int j = blockIdx.y * blockDim.y + threadIdx.y;
  int local_i = threadIdx.x;
  int local_j = threadIdx.y;

  float* a_shared = ;
  float* b_shared = ;

  /// CODE HERE (approx 20 lines) ///

}

Writing matmul_kernel.cu


In [43]:
!nvcc -c -o matmul_runner.o matmul_runner.cu
!nvcc -c -o matmul_kernel.o matmul_kernel.cu
!nvcc -o matmul matmul_runner.o matmul_kernel.o
!./matmul
!compute-sanitizer ./matmul

Matrix multiplication successful!
========= COMPUTE-SANITIZER
Matrix multiplication successful!
========= ERROR SUMMARY: 0 errors
